### Thoughts 

We set up a looper function - that runs through each labelled 30 second segment 
For (30 sec segment call X(x,y) where x is raw data and y is labe) :
    - feature extraction function that applies to it returns some vector/scalar of values (doubles) then appends on 
    This means that you can mix and match which features you want to be put into the loop 

At end of the loop you know have a vector of extracted features in the order that you placed the feature functions in with a label this is then saved to the feature extracted data file. 

In [ ]:
#import libraries 

In [ ]:
# Load the clean data you saved in the previous notebook.
#PROCESSED_DATA_PATH = 

### Function layout 
Each function should take in the pre processed EEG data, a vector representing where to store the features, append new features onto the end of the vector. 
The function will return the vector. 

In [ ]:
#Function for Extracting Hjforth parameters 

#Function for extracting spectral power 

#Function for extracting signal kurtosis 

#Function for extracting spectral entropy 

#Function for Extracting Band power rations 

In [ ]:
#Feature engineering and extraction 


#save extracted features as a vector with labelling. 